### 參考

aiohttp scraper: https://github.com/gaojiuli/gain  
tweet熱度：https://github.com/shirosaidev/stocksight  
reuters scraper: https://github.com/wenyenwei/tensorflow-stock-news  
https://stocknewsapi.com/  
https://stackoverflow.com/questions/51537063/url-format-for-google-news-rss-feed  
https://github.com/niderhoff/nlp-datasets

In [1]:
# https://github.com/LIAAD/yake
# !pip install nest_asyncio newspaper3k feedparser psycopg2
import nest_asyncio
nest_asyncio.apply()

# import nltk
# nltk.download('punkt')

# twstock

In [33]:
# !pip install twstock
import twstock
twstock.__update_codes()

In [36]:
# twstock.codes
stock = twstock.Stock('2330')
# stock.fetch_from(2010, 1)

ConnectionError: HTTPConnectionPool(host='www.twse.com.tw', port=80): Max retries exceeded with url: /exchangeReport/STOCK_DAY?date=20200401&stockNo=2330 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f8efe4989d0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [38]:
%cd /workspace/twint/app
import pandas as pd
import random

# df = pd.DataFrame(stock.data)
# df.to_csv("tw-2330.csv", index=False)

data = pd.read_csv('./resource/proxies.txt', sep=" ", header=None)
proxies = list(data[0])
p = random.choice(proxies)
p.split(':')

/workspace/twint/app


['104.247.159.138', '12345', 'cwang', '19861212']

# Newspaper3k

In [6]:
import newspaper

cnbc = newspaper.build('https://www.cnbc.com/markets/')

In [10]:
for a in cnbc.articles:
    print(a)

In [1]:
import newspaper

# url = "https://www.cnbc.com/2020/04/20/oil-markets-us-crude-futures-in-focus-as-coronavirus-dents-demand.html"
url = "https://news.cnyes.com/news/id/4484825"
# url = "https://tw.stock.yahoo.com/news/%E5%8F%B0%E5%8C%97%E8%82%A1%E5%B8%82-%E6%95%A3%E6%88%B6%E8%B3%87%E9%87%91%E5%9B%9E%E7%B1%A0-%E6%8D%A7%E7%B4%85%E9%8A%85%E6%9D%BF%E8%82%A1-083011251.html"
a = newspaper.Article(url=url)
a.download()
a.parse()
a.meta_data

Building prefix dict from /usr/local/lib/python3.7/site-packages/jieba/dict.txt ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.695194959640503 seconds.
Prefix dict has been built succesfully.


defaultdict(dict,
            {'fb': {'app_id': 325242790993768,
              'pages': '107207125979624,341172005895696,316306791736155,659317060868369,586654018059088'},
             'og': {'site_name': 'Anue鉅亨',
              'locale': 'zh_TW',
              'title': '經濟重啟降低避險需求 高盛開始做空美元 | Anue鉅亨 - 美股',
              'description': '隨著新冠肺炎 (COVID-19) 疫情放緩，各國政府開始逐步重啟商業活動，市場預期經濟有望溫和復甦，因此使得投資人對於美元的避險需求亦隨之降低，促使高盛開始建立做空美元的部位。',
              'type': 'article',
              'url': 'https://news.cnyes.com/news/id/4484825',
              'image': {'identifier': 'https://cimg.cnyes.cool/prod/news/4484825/l/836fe06fed84d39ad52c72730badccdc.jpg',
               'width': 640,
               'height': 360}},
             'description': '隨著新冠肺炎 (COVID-19) 疫情放緩，各國政府開始逐步重啟商業活動，市場預期經濟有望溫和復甦，因此使得投資人對於美元的避險需求亦隨之降低，促使高盛開始建立做空美元的部位。',
             'article': {'publisher': 'https://www.facebook.com/cnYES',
              'published_time': '2020-06-02T00:04:41+08:00',
              'author': 'https://www.

In [36]:
from lxml import etree

for e in a.clean_top_node.cssselect('a'):
    print(etree.tostring(e))

# for p in set([e.getparent() for e in node.cssselect('a')]):
#     text = etree.tostring(
#         p, method='text', encoding='utf-8').decode('utf-8').strip()
#     tk = _Ticker(text)

#     for a in p.cssselect('a'):
#         href = a.get('href')
#         queries = dict(urllib.parse.parse_qsl(
#             urllib.parse.urlsplit(href).query))
#         try:
#             tk.labels.append((a.text, queries['symbol']))
#         except KeyError:
#             continue
#     if len(tk.labels) > 0:
#         tickers.append(tk)

b'<a href="https://invest.cnyes.com/twstock/tws/2301">2301-TW</a>) &#20170; (9) &#26085;&#20659;&#20986;&#26071;&#19979;&#24037;&#26989;&#33258;&#21205;&#21270;&#20107;&#26989;&#37096;&#35009;&#21729;&#30334;&#20154;&#65292;&#23565;&#27492;&#65292;&#20809;&#23542;&#31185;&#28548;&#28165;&#65292;&#21482;&#26159;&#22240;&#25033;&#32771;&#26680;&#65292;&#36914;&#34892;&#30340;&#32068;&#32340;&#20839;&#37096;&#24120;&#24907;&#24615;&#35519;&#25972;&#65292;&#20197;&#20839;&#36681;&#28858;&#20778;&#20808;&#65292;&#20006;&#38750;&#22914;&#22806;&#20659;&#25152;&#35498;&#30340;&#35009;&#21729;&#19978;&#30334;&#20154;&#12290;'


In [27]:
import pandas as pd
import numpy as np

df = pd.read_csv('/workspace/twint/app/resource/rss_yahoo_tw.csv')
# return df.iterrows()
for index, r in df.iterrows():
    print(type(r['url']))    
    print(type(r['ticker']))    
    print(np.isnan(r['ticker']))
    print(r['ticker'], r['url'])
    
# pd.DataFrame({
#     "url": [1,2,3,4]
# })


<class 'str'>
<class 'float'>
True
nan https://tw.stock.yahoo.com/rss/url/d/e/N2.html
<class 'str'>
<class 'float'>
True
nan https://tw.stock.yahoo.com/rss/url/d/e/N3.html
<class 'str'>
<class 'float'>
True
nan https://tw.stock.yahoo.com/rss/url/d/e/N4.html
<class 'str'>
<class 'float'>
True
nan https://tw.stock.yahoo.com/rss/url/d/e/N5.html
<class 'str'>
<class 'float'>
True
nan https://tw.stock.yahoo.com/rss/url/d/e/N6.html
<class 'str'>
<class 'float'>
True
nan https://tw.stock.yahoo.com/rss/url/d/e/N7.html
<class 'str'>
<class 'float'>
True
nan https://tw.stock.yahoo.com/rss/url/d/e/N8.html
<class 'str'>
<class 'float'>
True
nan https://tw.stock.yahoo.com/rss/url/d/e/N9.html
<class 'str'>
<class 'float'>
True
nan https://tw.stock.yahoo.com/rss/url/d/e/N10.html
<class 'str'>
<class 'float'>
True
nan https://tw.stock.yahoo.com/rss/url/d/e/N11.html
<class 'str'>
<class 'float'>
True
nan https://tw.stock.yahoo.com/rss/url/d/e/N12.html
<class 'str'>
<class 'float'>
True
nan https://tw.s

# aiohttp

In [2]:
import aiohttp
from newspaper import Article

url = 'http://cnb.cx/sytyjc'
# url = 'https://www.cnbc.com/2020/04/19/why-big-techs-coronavirus-goodwill-wont-help-in-antitrust-probes.html'

async def get():
    async with aiohttp.ClientSession() as sess:
        async with sess.get(url) as resp:
#             print(resp)
#             print(resp.status)
#             print(resp.headers)
#             print(resp.url)
#             print(resp.history)
            html = await resp.text()
#             article = Article(str(resp.url))
#             article.set_html()
#             article.parse()
            return resp, html

# article, resp = await get()
# print(article)
try:
    resp, html = await get()
except aiohttp.ClientError as e:
    print(e)

In [6]:
try:
    try:
        raise error()
    except IOError as e:
        print(e)
except Exception as e:
    print(e)
finally:
    print('aa')

name 'error' is not defined
aa


In [11]:
import re

p = re.compile(r'^\/tag\/(\w+)') 
m = p.search('/tag/') 
# print(m.group())
print(m)
# m.group(1)
if m is not None:
    print(m.group(1))

None


In [48]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')
for link in soup.find_all('a'):
    href = link.get('href')
    if href is not None and '/tag/' in href:
        print(href)
#     print(link.get('href'))
    

/tag/光寶科
/tag/LED
/tag/資訊
/tag/電源
/tag/裁員


In [28]:
import asyncio
import aiohttp
from fake_useragent import UserAgent

# url = "https://www.cnbc.com/id/45044666" # 404
urls = [
    "https://httpstat.us/404",
    "https://httpstat.us/200",
]

async def get(url):
    ua = UserAgent(verify_ssl=False, use_cache_server=False).random
    async with aiohttp.ClientSession(raise_for_status=True, headers=[("User-Agent", ua)]) as sess:
            async with sess.get(url) as resp:
                return resp


# await asyncio.gather(*[get(u) for u in urls])


    
try:
    resp = await get(urls[1])
    print(resp.status)
    print(type(resp.status))
except aiohttp.ClientResponseError as e:
    print(e)
    print(type(e.status))
    print(e.headers)
    print(e.request_info.real_url)
    print(e.request_info)

200
<class 'int'>


In [137]:
import json
# article.meta_keywords
# article.meta_site_name
# s = json.dumps(article.meta_data)
# json.loads(s)
# article.article_html
# article.source_url
type(article.meta_data)

collections.defaultdict

In [2]:
from lxml import etree

article.clean_top_node

<Element div at 0x7f81a24329b0>

In [120]:
from urllib import parse
import urllib
from lxml import etree
# from bs4 import BeautifulSoup

# resp
# resp.headers
article.meta_data
# type(article.top_node)
# tostring(article.top_node)

# soup = BeautifulSoup(article.top_node, 'lxml')
# BeautifulSoup(doc, 'lxml')
# soup

# type(article.top_node)
# print(etree.tostring(article.top_node, pretty_print=True).decode('utf-8'))
# print(etree.tostring(article.clean_top_node, pretty_print=True).decode('utf-8'))
# print('this is a \'try')

# for e in article.clean_top_node.cssselect('a'):
#     print(etree.tostring(e))
#     print(e.getparent().text)
#     print(e.text)
#     print(e.get('href'))
#     print(e.keys())

# es = article.clean_top_node.cssselect('a')
# href = e.get('href')
# print(dict(urllib.parse.parse_qsl(urllib.parse.urlsplit(href).query)))
# print(e.text)
# print(etree.tostring(e.getparent(), method='text', pretty_print=True, encoding='utf-8').decode('utf-8'))
# print(e.getparent().text)
# print(etree.tostring(article.clean_top_node, pretty_print=True).decode('utf-8'))

# es[0].getparent() is es[1].getparent()
# set([es[0].getparent(), es[1].getparent()])
# e = es[0].getparent()
# # e.cssselect('a')
# print(etree.tostring(e, method='text', encoding='utf-8').decode('utf-8').strip())
# for a in e.cssselect('a'):
#     print(etree.tostring(a, with_tail=False))
#     print(a.text)
#     print(a.items())

import dataclasses
from typing import Tuple, List


@dataclasses.dataclass
class _TickerAnchor:
    text: str
    ticker: str


@dataclasses.dataclass
class ParsedTicker:
    '''<p>...some text...<a href='$AAA'>...some anchor text</a>...<a>....</a>...</p>
    '''
    text: str
    tickers: List[_TickerAnchor] = dataclasses.field(default_factory=list)


node =  article.clean_top_node

data = []
for p in set([e.getparent() for e in node.cssselect('a')]):
    text = etree.tostring(
        p, method='text', encoding='utf-8').decode('utf-8').strip()
    parsed = ParsedTicker(text)
#     print(parsed)
    for a in p.cssselect('a'):
#         etree.tostring(a)
        href = a.get('href')
        queries = dict(urllib.parse.parse_qsl(
            urllib.parse.urlsplit(href).query))
        try:
            parsed.tickers.append(
                _TickerAnchor(a.text, queries['symbol'])
            )
        except KeyError:
            continue
    if len(parsed.tickers) > 0:
        data.append(parsed)
        print(parsed)

# data[2]

ParsedTicker(text="Recent actions by both agencies back up their claims. At the end of March, the DOJ Antitrust Division announced it would require military technology companies Raytheon and United Technologies Corporation to divest parts of their businesses to proceed with their proposed merger. On April 1, the FTC announced its lawsuit to unwind tobacco company Altria's $12.8 billion investment in Juul, the e-cigarette company accused of luring adolescents to its products with its marketing.", tickers=[_TickerAnchor(text='Raytheon ', ticker='UTX'), _TickerAnchor(text='Altria', ticker='MO')])
ParsedTicker(text='Up until recently, Facebook, Google, Amazon and Apple had been met with unyielding scrutiny in Washington and most state attorneys general offices, barraged by questions of dominance and privacy violations. Facebook and Google have been most visibly pursued by federal authorities and state attorneys.', tickers=[_TickerAnchor(text='Facebook', ticker='FB'), _TickerAnchor(text='Go

In [50]:
from urllib import parse
import urllib
url = "/quotes/?symbol=FB"
# url = "http://www.example.org/default.html?ct=32&op=92&item=98"
print(urllib.parse.urlsplit(url))
# parse.parse_qs(parse.urlsplit(url).query)
dict(urllib.parse.parse_qsl(urllib.parse.urlsplit(url).query))

SplitResult(scheme='', netloc='', path='/quotes/', query='symbol=FB', fragment='')


{'symbol': 'FB'}

# asyncio

In [8]:
import nest_asyncio
nest_asyncio.apply()
import asyncio
import random
import time

import asyncio
import random
import time


async def worker(name, queue):
    while True:
        sleep_for = await queue.get()
        try:
            try:
                await asyncio.sleep(sleep_for)
                print(aaa)
            except IOError:
                pass
        except Exception as e:
            print(e)
        finally:
            queue.task_done()
            print(f'{name} has slept for {sleep_for:.2f} seconds')


async def main():
    # Create a queue that we will use to store our "workload".
    queue = asyncio.Queue()

    # Generate random timings and put them into the queue.
    total_sleep_time = 0
    for _ in range(20):
        sleep_for = random.uniform(0.05, 1.0)
        total_sleep_time += sleep_for
        queue.put_nowait(sleep_for)

    # Create three worker tasks to process the queue concurrently.
    tasks = []
    for i in range(3):
        task = asyncio.create_task(worker(f'worker-{i}', queue))
        tasks.append(task)

    # Wait until the queue is fully processed.
    started_at = time.monotonic()
    await queue.join()
    total_slept_for = time.monotonic() - started_at

    # Cancel our worker tasks.
    for task in tasks:
        task.cancel()
    # Wait until all worker tasks are cancelled.
    await asyncio.gather(*tasks, return_exceptions=True)

    print('====')
    print(f'3 workers slept in parallel for {total_slept_for:.2f} seconds')
    print(f'total expected sleep time: {total_sleep_time:.2f} seconds')


asyncio.run(main())

name 'aaa' is not defined
worker-2 has slept for 0.18 seconds
name 'aaa' is not defined
worker-0 has slept for 0.45 seconds
name 'aaa' is not defined
worker-1 has slept for 0.61 seconds
name 'aaa' is not defined
worker-0 has slept for 0.29 seconds
name 'aaa' is not defined
worker-0 has slept for 0.23 seconds
name 'aaa' is not defined
worker-2 has slept for 0.78 seconds
name 'aaa' is not defined
worker-1 has slept for 0.46 seconds
name 'aaa' is not defined
worker-1 has slept for 0.36 seconds
name 'aaa' is not defined
worker-2 has slept for 0.67 seconds
name 'aaa' is not defined
worker-0 has slept for 0.71 seconds
name 'aaa' is not defined
worker-0 has slept for 0.36 seconds
name 'aaa' is not defined
worker-0 has slept for 0.26 seconds
name 'aaa' is not defined
worker-1 has slept for 0.96 seconds
name 'aaa' is not defined
worker-2 has slept for 0.98 seconds
name 'aaa' is not defined
worker-1 has slept for 0.48 seconds
name 'aaa' is not defined
worker-2 has slept for 0.26 seconds
name 'aa

In [10]:
from fake_useragent import UserAgent

ua = UserAgent(verify_ssl=False)
ua.update()

ua.random

'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/30.0.1599.17 Safari/537.36'

In [22]:
# ua.random
ua.random

'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36'

# feedparser, rss

In [1]:
import requests

proxies = {"http": "http://173.234.248.57:3128", "https": "http://173.234.248.57:3128"}
# proxies = {"http": "http://192.161.163.206:3128", "https": "http://192.161.163.206:3128"}
r = requests.get('http://finance.yahoo.com/', proxies=proxies)

In [2]:
r.status_code
# r.text
r.headers

{'Referrer-Policy': 'no-referrer-when-downgrade', 'Strict-Transport-Security': 'max-age=15552000', 'X-Frame-Options': 'SAMEORIGIN', 'Content-Security-Policy': 'sandbox allow-downloads allow-forms allow-modals allow-same-origin allow-scripts allow-popups allow-popups-to-escape-sandbox allow-top-navigation-by-user-activation allow-presentation;', 'Content-Type': 'text/html; charset=utf-8', 'Content-Encoding': 'gzip', 'Set-Cookie': 'B=29hqlftfeklo9&b=3&s=l5; expires=Wed, 17-Jun-2021 17:46:49 GMT; path=/; domain=.yahoo.com', 'Date': 'Wed, 17 Jun 2020 17:46:49 GMT', 'Server': 'ATS', 'Cache-Control': 'max-age=0, private', 'Expires': '-1', 'Age': '0', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Expect-CT': 'max-age=31536000, report-uri="http://csp.yahoo.com/beacon/csp?src=yahoocom-expect-ct-report-only"', 'X-XSS-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff'}

In [6]:
# # !pip install aiohttp_proxy
# import aiohttp
# from aiohttp_proxy import ProxyConnector, ProxyType

# async def fetch():
#     connector = ProxyConnector.from_url('http://173.234.248.57:3128')
#     ### or use ProxyConnector constructor
# #     connector = ProxyConnector(
# #         proxy_type=ProxyType.SOCKS5,
# #         host='173.234.232.170',
# #         port=3128,
# # #         username='user',
# # #         password='password',
# #         rdns=True
# #     )
#     async with aiohttp.ClientSession(connector=connector) as session:
#         async with session.get("https://finance.yahoo.com") as resp:
#             print(resp.status)
#             return await response.text()

# await fetch()

In [28]:
import aiohttp
import feedparser

headers = {
#     "If-None-Match": "246ce00049b1df9930d4dc344180574f"
}

async def get():
    async with aiohttp.ClientSession() as session:
#         async with session.get('http://finance.yahoo.com/rss/industry?s=yhoo', headers=headers) as resp:
        proxy_auth = aiohttp.BasicAuth('cwang', '19861212')
        async with session.get('http://finance.yahoo.com/rss/industry?s=yhoo',
                               proxy="http://107.174.248.115:12345",
                               proxy_auth=proxy_auth,
                               headers=headers) as resp:
            print(resp.status)
            print(resp.headers)
            t = await resp.text()
            return feedparser.parse(t)

await get()

200
<CIMultiDictProxy('x-amz-id-2': 'aFr/4c38xkbXOn6VTFwYt0/AitPbsxn+8jtIITIx4NHZcfRMaXSJhnzAF+LqhrC4h+YtOhZd95Y=', 'x-amz-request-id': 'AB23E8C9C7272BA8', 'Date': 'Tue, 07 Jul 2020 14:49:45 GMT', 'Last-Modified': 'Tue, 07 Jul 2020 14:48:10 GMT', 'Etag': '"32676ab4b04edb2d5263c897cf5de533"', 'x-amz-server-side-encryption': 'AES256', 'Cache-Control': 'public, max-age=600', 'Accept-Ranges': 'bytes', 'Content-Type': 'application/xml', 'Content-Length': '255831', 'Server': 'ATS', 'Referrer-Policy': 'no-referrer-when-downgrade', 'Vary': 'Origin', 'Age': '248', 'Strict-Transport-Security': 'max-age=15552000', 'X-Frame-Options': 'SAMEORIGIN', 'Connection': 'keep-alive', 'Expect-CT': 'max-age=31536000, report-uri="http://csp.yahoo.com/beacon/csp?src=yahoocom-expect-ct-report-only"', 'X-XSS-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff')>


{'feed': {'title': 'Yahoo Finance',
  'title_detail': {'type': 'text/plain',
   'language': None,
   'base': '',
   'value': 'Yahoo Finance'},
  'links': [{'rel': 'alternate',
    'type': 'text/html',
    'href': 'https://finance.yahoo.com/'}],
  'link': 'https://finance.yahoo.com/',
  'subtitle': 'At Yahoo Finance, you get free stock quotes, up-to-date news, portfolio management resources, international market data, social interaction and mortgage rates that help you manage your financial life.',
  'subtitle_detail': {'type': 'text/html',
   'language': None,
   'base': '',
   'value': 'At Yahoo Finance, you get free stock quotes, up-to-date news, portfolio management resources, international market data, social interaction and mortgage rates that help you manage your financial life.'},
  'language': 'en-US',
  'rights': 'Copyright (c) 2020 Yahoo! Inc. All rights reserved',
  'rights_detail': {'type': 'text/plain',
   'language': None,
   'base': '',
   'value': 'Copyright (c) 2020 Ya

In [2]:
import feedparser
f = feedparser.parse('https://tw.stock.yahoo.com/rss/url/d/e/N2.html')

In [3]:
# import json
# import pprint

# # pprint.pprint(f['entries'][0])
# # j = json.dumps(f['entries'][0])
# entries = json.loads(a)
# len(entries)
# entries[0]

# f['entries'][0]
# f
f["entries"][0]

{'title': '《各報要聞》川普出招制裁陸！撤銷香港特殊待遇 宣布退出世衛',
 'title_detail': {'type': 'text/plain',
  'language': None,
  'base': 'https://tw.stock.yahoo.com/rss/url/d/e/N2.html',
  'value': '《各報要聞》川普出招制裁陸！撤銷香港特殊待遇 宣布退出世衛'},
 'links': [{'rel': 'alternate',
   'type': 'text/html',
   'href': 'https://tw.stock.yahoo.com/news/各報要聞-川普出招制裁陸-撤銷香港特殊待遇-宣布退出世衛-005758046.html'}],
 'link': 'https://tw.stock.yahoo.com/news/各報要聞-川普出招制裁陸-撤銷香港特殊待遇-宣布退出世衛-005758046.html',
 'published': 'Sun, 31 May 2020 08:57:58 CST',
 'published_parsed': time.struct_time(tm_year=2020, tm_mon=5, tm_mday=31, tm_hour=14, tm_min=57, tm_sec=58, tm_wday=6, tm_yday=152, tm_isdst=0),
 'id': '',
 'guidislink': False,
 'summary': '【時報-台北電】大陸全國人大28日通過港版「國安法」後，美國總統川普當地時間29日下午召開記者會，他指責大陸破壞對香港及世界承諾，而作為懲罰陸方措施的一環，美國將撤銷給予香港的特殊待遇，包括單獨關稅區...',
 'summary_detail': {'type': 'text/html',
  'language': None,
  'base': 'https://tw.stock.yahoo.com/rss/url/d/e/N2.html',
  'value': '【時報-台北電】大陸全國人大28日通過港版「國安法」後，美國總統川普當地時間29日下午召開記者會，他指責大陸破壞對香港及世界承諾，而作為懲罰陸方措施的一環，美

In [30]:
import datetime
import json
t = [2020, 4, 14, 21, 35, 43, 1, 105, 0]

# datetime.datetime.now().timestamp()
# datetime.datetime.fromtimestamp(1588500008)

# d = datetime.timedelta(days=30)
# until - d

# json.dumps(datetime.datetime(*t[:7]))

def startpoints(start: datetime.datetime, until: datetime.datetime):
    urls = []
    while start < until:
        _until = start + datetime.timedelta(days=30)
        if _until > until:
            _until = until
        urls.append('https://news.cnyes.com/api/v3/news/category/headline' +
                    f'?startAt={int(start.timestamp())}' +
                    f'&endAt={int(_until.timestamp())}' +
                    '&limit=100')
        start = _until
    return urls

start = datetime.datetime(2010, 1, 1)
until = datetime.datetime.now()
print(until.timestamp())
print(startpoints(start, until))

1588509731.906466
['https://news.cnyes.com/api/v3/news/category/headline?startAt=1262304000&endAt=1264896000&limit=100', 'https://news.cnyes.com/api/v3/news/category/headline?startAt=1264896000&endAt=1267488000&limit=100', 'https://news.cnyes.com/api/v3/news/category/headline?startAt=1267488000&endAt=1270080000&limit=100', 'https://news.cnyes.com/api/v3/news/category/headline?startAt=1270080000&endAt=1272672000&limit=100', 'https://news.cnyes.com/api/v3/news/category/headline?startAt=1272672000&endAt=1275264000&limit=100', 'https://news.cnyes.com/api/v3/news/category/headline?startAt=1275264000&endAt=1277856000&limit=100', 'https://news.cnyes.com/api/v3/news/category/headline?startAt=1277856000&endAt=1280448000&limit=100', 'https://news.cnyes.com/api/v3/news/category/headline?startAt=1280448000&endAt=1283040000&limit=100', 'https://news.cnyes.com/api/v3/news/category/headline?startAt=1283040000&endAt=1285632000&limit=100', 'https://news.cnyes.com/api/v3/news/category/headline?startAt=1

In [23]:
from time import mktime
import datetime
 
fetched_at = datetime.datetime(2020, 4, 12, 10, 0, 0)
freq = datetime.timedelta(seconds=12000)
print(fetched_at + freq)
secs_to_sleep = (fetched_at + freq - datetime.datetime.now()).total_seconds()

if secs_to_sleep > 0:
    print("sleep for {} seconds".format(int(secs_to_sleep)))
else:
    print("scrape")

2020-04-12 13:20:00
sleep for 5955 seconds


# twint, elasticsearch

In [28]:
import twint

c = twint.Config()

# c.Username = "YahooFinance"
# c.Username = "CNBC"
c.Search = '$AAPL'
c.Limit = 10
# c.Since= '2017-12-27'
# c.Until = '2017-12-31'
# c.Store_csv = True
# c.Output = "none"
c.Elasticsearch = "http://es:9200"
c.Debug = True

twint.run.Search(c)

[+] Indexing to Elasticsearch @ http://es:9200
....................

In [6]:
from lxml import etree

# html = etree.tostring(article.clean_top_node, encoding='utf-8').decode('utf-8')
# e = etree.fromstring(html)
# etree.tostring(e)
# print(etree.tostring(article.top_node)[:300])
# print(etree.tostring(article.clean_top_node)[:300])
import logging
import sys

log = logging.getLogger("elasticsearch")
log.addHandler(logging.StreamHandler(sys.stdout))

In [7]:
import elasticsearch
from elasticsearch_dsl import Q
from app.store import es

es.init()
# client = elasticsearch.Elasticsearch(['es:9200'])
# client.indices.delete(index="news_rss")
# client.indices.delete(index="news_page")

def f():
    s = es.Page.search()
#     s = s.filter('wildcard', from_url="*cnyes*").filter('term', http_status="*cnyes*")
    q = Q('wildcard', from_url="*cnyes.com*") \
            & ~Q("term", http_status=200)
    print(q)
    s = s.filter(q)
    
    results = s.execute()
    for page in results:
        print(page.from_url, page.http_status)
    # s.scan()
#     for p in s.scan():
#         yield p.from_url

# for x in f():
#     print(x)
#     break
# f()

In [5]:
from app.store import es

p = es.Page(from_url="aaa.com")
p.__dict__

{'meta': {}, '_d_': {'from_url': 'aaa.com'}}

In [40]:
import json
import datetime
from lxml import etree
import elasticsearch
from elasticsearch_dsl import connections, Document, Date, Keyword, Q, Search, Text, Range, Integer

class Rss(Document):
    url = Keyword(required=True)
    ticker = Keyword()
    freq = Integer()  # seconds (default: 2 days)
    n_retries = Integer()
    fetched_at = Date()

    class Index:
        name = "news_rss"
        settings = {
            'number_of_shards': 1,
            'number_of_replicas': 0
        }

    def save(self, **kwargs):
        self.meta.id = self.url
        self.n_retries = self.n_retries or 0
        self.freq = self.freq or 2*24*60*60
        # self.created_at = datetime.datetime.now()
        return super().save(**kwargs)
    
    @classmethod
    def get_or_create(cls, url) -> Rss:
        try:
            rss = cls.get(id=url)
        except elasticsearch.NotFoundError:
            rss = cls(url=url)
            rss.save()
        return rss

connections.create_connection(hosts=['es:9200'])
Rss.init()

# rss = Rss(url="https://some.url.com/?query=like&this")
# rss.save()

r = Rss.get_or_create("https://some.url.com/?query=like&thisad")
# print(r.n_retries)
print(r)


# for i, hit in enumerate(scan_twint('ReutersBiz')):
#     print(hit.urls)
#     if i > 10:
#         break

# connections.create_connection(hosts=['es:9200'])
# connections.create_connection(hosts=['es:9200'])
# Page.init()
# s = Page.search()
# s.query = Q({"match": {"src_url": "aaa.com"}})
# resp = s.execute()
# print(resp.hits.total.value)
# Page.is_existed('aaav.com')




Rss(index='news_rss', id='https://some.url.com/?query=like&thisad')


In [20]:
import elasticsearch
from elasticsearch_dsl import connections, Document, Date, Keyword, Q, Search, Text, Range, Integer

q = Q({
        "range": {
            "date": {
                "gte": "2000-01-01 00:00:00",
                "lt": "2025-01-01 00:00:00",
            }
        }
    })
client = elasticsearch.Elasticsearch(['es:9200'])
s = Search(using=client, index="twinttweets").query(q).filter("terms", username=["CNBC"])

# for hit in s.scan():
#     print(hit)
hit = next(s.scan())
hit.username
hasattr(hit, "urls")

True

In [16]:
# dir(hit.username)
hit.username

'ReutersBiz'

# @dataclass, namedtuple

In [79]:
import json
import dataclasses 
from typing import Tuple, List

@dataclasses.dataclass
class _TickerAnchor:
    a_text: str
    ticker: str

@dataclasses.dataclass
class ParsedTicker:
    p_text: str  # <p>...text...</p>
    tickers: List[_TickerAnchor]
    

t = ParsedTicker('some p text', [_TickerAnchor('some a text', 'AAA')])
s = json.dumps(dataclasses.asdict(t))
ParsedTicker(**json.loads(s))

ParsedTicker(p_text='some p text', tickers=[{'a_text': 'some a text', 'ticker': 'AAA'}])

In [ ]:
from collections import namedtuple

ParsedTicker = namedtuple('ParsedTicker', ['p_text', 'symbols'])
Symbol = namedtuple('Symbol', ['a_text', 'symbol'])

Symbol

In [3]:
%cd /workspace/twint/app
from app.store import es

es.init()
p = es.Page(from_url="aaa.com")
p.save()
# p.meta.id = 'aaa'
print(p)
# p.meta.id = 'bbb'
# p.meta.id
# p.save()
# type(p.meta)
# p.meta.id = 'aaa.com'
# p.save()
# p.created_at is None

# p = es.Page.get(id="aaa.com")
# p.meta.id='aaa'
# p.save()

/workspace/twint/app
Page(index='news_page', id='aaa.com')


In [31]:
a = [1,2,3]
b = set(a)
b.add(4)
b
b

{1, 2, 3, 4}

In [25]:
from get_all_tickers.get_tickers import get_tickers
save_to = "./rss_yahoo_us.csv"
tickers = get_tickers()
urls = [
    "http://finance.yahoo.com/rss/headline?s={}".format(t) for t in tickers]
df = pd.DataFrame({
    "ticker": tickers,
    "url": urls,
})
df.to_csv(save_to, index=False)
